In [1]:
import numpy as np
import torch

In [2]:
import matplotlib
from matplotlib import pyplot as plt
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

In [3]:
pt_path1 = '../results/sw/sw_CAE_h32_k5_ReZero_default_none_none/sw_CAE_h32_k5_ReZero_default_none_none_sigma_z_b=0.001_sigma_m=0.003_method=ETKF-Q_assimilated.pt'
pt_path2 = '../results/sw/sw_AEflow_b4_k5_ReZero_default_none_none/sw_AEflow_b4_k5_ReZero_default_none_none_sigma_z_b=0.001_sigma_m=0.1_method=ETKF-Q_assimilated.pt'
pt_path3 = '../results/sw/sw_SINRv11_default_NeuralODE_default_none_none/sw_SINRv11_default_NeuralODE_default_none_none_sigma_z_b=0.03_sigma_m=0.003_method=DEnKF_assimilated.pt'

In [4]:
ass_data1 = torch.load(pt_path1)
ass_data2 = torch.load(pt_path2)
ass_data3 = torch.load(pt_path3)

ass_data1

{'method': 'ETKF-Q',
 'ass_data': {'x_b': tensor([-0.1662, -7.1940, -7.2120,  ..., 28.8008, 18.0036, 21.9988]),
  'covB': tensor([[1.0000e-06, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 1.0000e-06, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 1.0000e-06,  ..., 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.0000e-06, 0.0000e+00,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 1.0000e-06,
           0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
           1.0000e-06]]),
  'yy_o': tensor([[ 0.7676, -0.8113,  0.7544,  ...,  2.3581,  0.0585, -0.4401],
          [ 0.6110, -0.6102,  0.8887,  ...,  2.2399, -0.1583, -0.2660],
          [ 0.8595, -0.4821,  0.7940,  ...,  2.3072, -0.0731, -0.4211],
          ...,
          [ 0.8952

In [6]:
import sys

sys.path.append('../')
from datasets import MetaData

dataset_ts_path = '/home/lizhuoyuan/datasets/shallow_water/cached_meta_ts.pt'
dataset_ts:MetaData = torch.load(dataset_ts_path)
dataset_ts.trajs[0]

MetaData(trajs=tensor([[[[[-0.8332,  0.1319],
           [-0.8577,  0.1609],
           [-0.8864,  0.1843],
           ...,
           [-0.9021, -0.2139],
           [-0.8715, -0.1767],
           [-0.8449, -0.1547]],

          [[-0.8336,  0.1323],
           [-0.8594,  0.1630],
           [-0.8902,  0.1944],
           ...,
           [-0.9075, -0.2327],
           [-0.8742, -0.1813],
           [-0.8458, -0.1557]],

          [[-0.8339,  0.1327],
           [-0.8610,  0.1651],
           [-0.8940,  0.2057],
           ...,
           [-0.9128, -0.2540],
           [-0.8768, -0.1864],
           [-0.8467, -0.1567]],

          ...,

          [[-0.8321,  0.1305],
           [-0.8522,  0.1548],
           [-0.8743,  0.1619],
           ...,
           [-0.8857, -0.1738],
           [-0.8631, -0.1650],
           [-0.8419, -0.1513]],

          [[-0.8325,  0.1310],
           [-0.8541,  0.1568],
           [-0.8783,  0.1681],
           ...,
           [-0.8911, -0.1845],
           [-